In [1]:
from sklearn.datasets import load_boston
import sklearn.ensemble
import numpy as np
import pandas as pd

In [2]:
boston = load_boston()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [3]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(boston.data, boston.target, train_size=0.80, random_state=42)

In [4]:
#from sklearn.neural_network import MLPRegressor
#nn = MLPRegressor(solver='lbfgs', hidden_layer_sizes=(8,8), random_state=0, max_iter=1000)
import xgboost
rf = xgboost.XGBRegressor()
rf.fit(train, labels_train)
rf2 = xgboost.XGBRegressor()

[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [5]:
MAE = np.mean(abs(rf.predict(test) - labels_test))
print('Random Forest Mean abusolute error', MAE)

Random Forest Mean abusolute error 1.9809417677860635


In [6]:
from scipy import stats
outlier_bool = ((np.abs(stats.zscore(train)) < 3).all(axis=1)) == 0     # (zscore < 3) == 0 means false, means outliers

In [7]:
outlier_index = [i for i, x in enumerate(outlier_bool) if x]

In [8]:
influential_index = 0
max_influence = 0
for i in outlier_index:
  train_without = train.tolist()
  train_without.pop(i)
 
  labels_train_without = labels_train.tolist()
  labels_train_without.pop(i)
  
  model_without = rf2.fit(train_without, labels_train_without)
  
  influence = np.mean(abs(rf.predict(test) - model_without.predict(test)))
  print('influence of instance', i, ': ', influence)
  print('Score: ', model_without.score(test,labels_test))
  temp_influence = influence
  if max_influence < temp_influence:
    max_influence = temp_influence
    influential_index = i
print('Influential instance is instance: ', influential_index, '. Influence is:', max_influence)

[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
influence of instance 3 :  0.40661007
Score:  0.9012554971287479
[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
influence of instance 7 :  0.9985618
Score:  0.8947295937948165
[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
influence of instance 8 :  0.68809736
Score:  0.8947652537289993
[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
influence of instance 20 :  0.6270958
Score:  0.8945997550226192
[02:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
influence of instance 24 :  0.6613886
Score:  0.9051855971332528
[02:09:43] WARNING: /workspace/src/object

In [9]:
###
# checking all datapoints for influential instances


for i in range(len(train)-1):
  train_without = train.tolist()
  train_without.pop(i)
 
  labels_train_without = labels_train.tolist()
  labels_train_without.pop(i)
  
  model_without = rf2.fit(train_without, labels_train_without)
  print('influence of instance i', np.mean(abs(rf.predict(test) - model_without.predict(test))))


NameError: ignored